<a href="https://colab.research.google.com/github/WilhelmWillie/i-love-produce/blob/master/i_love_produce_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing our good old TensorFlow libraries

import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers

In [0]:
# Import os so we can do path stuff
import os 

base_dir = os.path.join("./training_data")

# Base directories for each carrot
carrot_dir = os.path.join(base_dir, "carrot")
onion_dir = os.path.join(base_dir, "onion")
tomato_dir = os.path.join(base_dir, "tomato")

# Specific directories  for training 
carrot_dir_train = os.path.join(carrot_dir, "train") 
carrot_dir_val = os.path.join(carrot_dir, "val")
onion_dir_train = os.path.join(onion_dir, "train")
onion_dir_val = os.path.join(onion_dir, "val")
tomato_dir_train = os.path.join(tomato_dir, "train")
tomato_dir_val = os.path.join(tomato_dir, "val")

print(base_dir, carrot_dir, onion_dir, tomato_dir)

./training_data ./training_data/carrot ./training_data/onion ./training_data/tomato


In [0]:
# Make each directory if they don't exist
try: 
  os.mkdir(carrot_dir_train)
  os.mkdir(carrot_dir_val)
  os.mkdir(onion_dir_train)
  os.mkdir(onion_dir_val)
  os.mkdir(tomato_dir_train)
  os.mkdir(tomato_dir_val)
except FileExistsError:
  print("Directories already exist... moving on")
except:
  print("Something else went wrong...")

Directories already exist... moving on


In [0]:
# Randomly shuffle files
import random 

carrot_files = os.listdir(carrot_dir)
onion_files = os.listdir(onion_dir)
tomato_files = os.listdir(tomato_dir)

random.shuffle(carrot_files)
random.shuffle(onion_files)
random.shuffle(tomato_files)

print(carrot_files, onion_files, tomato_files)

['2232745_web1_gtr-liv-garden02110418.jpg', 'images.jpg', '03116_01_romance.jpg', 'download.jpg', 'baby carrots 0225.jpg', 'ad2fb3c0-5e39-4e37-9189-b77942093687-119f71b.jpg', 'carrots-7-1200.jpg', 'ThinkstockPhotos-507126001_0.jpg', 'glazed-carrots-2.jpg', '1298630794-H.jpg', 'val', '1547589978038.jpg', 'carrots-table_popidar-ss.jpg', 'healthy-carrot-soup-image.jpg', 'Spend-With-Pennies-Roasted-Carrots-25-500x375.jpg', 'train', '0358A-scarlet-nantes-organic-carrot.jpg', 'prod000633.jpg', '5295-mans_hand_picking_up_a_bunch_of_carrots-732x549-thumbnail.jpg', 'unnamed.jpg', 'il-794xn-433776190-eylc-1566242511.jpg'] ['unnamed.jpg', '1452744949352.jpg', 'FQU_eUpdate_0925_onions.jpg', 'train', 'Onion-alternative_1200.jpg', 'Mixed_onions.jpg', '90021_sydavidson3_8f77de46-2be6-48bc-9ff9-a006839d390e.jpg', '02581_01_cabernet.jpg', 'Paradigm-Group.jpg', 'savannah-sweet-onion.jpg', 'how-to-cut-onion_hero.jpg', '81UeYuulNjL._SL1500_.jpg', 'types-of-onions-2-1200.jpg', 'onion_garlic_toxcicity.jpg',

In [0]:
# Split and copy files
import shutil

def split_and_copy_files(files, class_dir, train_dir, val_dir):
  train_files = files[:len(files)//2]
  val_files = files[len(files)//2:]

  for file in train_files:
    if os.path.isdir(os.path.join(class_dir, file)) is not True:
      shutil.copy(os.path.join(class_dir, file), os.path.join(train_dir, file))

  for file in val_files:
    if os.path.isdir(os.path.join(class_dir, file)) is not True:
      shutil.copy(os.path.join(class_dir, file), os.path.join(val_dir, file))

split_and_copy_files(carrot_files, carrot_dir, carrot_dir_train, carrot_dir_val)
split_and_copy_files(onion_files, onion_dir, onion_dir_train, onion_dir_val)
split_and_copy_files(tomato_files, tomato_dir, tomato_dir_train, tomato_dir_val)

val
train
train
val
train
val
